# Diagnosing Model Driven Telemetry timeseries

This notebook uses an LLM to turn a selected set of sensor paths (sensor path name and value) into a "diagnosis" in natural language.

In [ ]:
%load_ext autoreload
%autoreload 2

## Load MDT Changepoints
An MDT changepoint maps to a significant network state change detected within a network device, capturing the correlating ranked sensor-paths as features to perform a diagnosis.

For the purposes of this experiment, we start with a JSON file capturing a collection of MDT Changepoints, and for each changepoint, a list of MDT sensors paths, as features, grouped by the source router. This notebook focuses on the approach to  diagnose one router within a selected changepoint.



In [ ]:
import os
import json
from modules.logger import Logger
import modules.utils as utils

# returns JSON object as 
# a dictionary
changepoints = json.load(open('./datasets/mdt-demo.json'))
mdt_changepoint = changepoints[0]
mdt_changepoint['Features'] = '\n'.join(mdt_changepoint['Features'])

utils.displayDictionary(mdt_changepoint)


## Initialise LLM Client

In [ ]:
from modules.llm.azure_ai import AzureLlm
from dotenv import load_dotenv

load_dotenv("env")

logger = Logger()
llm = AzureLlm(logger,os.getenv('AZURE_OPENAI_API_KEY'))


## Perform Initial Diagnosis
Use a prompt engineered to perform an initial draft diagnosis from the MDT changepoint features extracted for a specific device.


### Prompt Engineering: Sensor Path Diagnosis Prompt

Implement a specific prompt engineered to request the LLM to perform a network level diagnosis based upon the extracted MDT sensor path features.

In [ ]:
from string import Template
from modules.llm.prompt import JsonCompletion
from modules.llm.llm import LlmInterface
    
class MdtDiagnosisPrompt(JsonCompletion):
    _template = Template(
"""You are a networking expert.
You are diagnosing a network issue based on telemetry information received from a Cisco router running IOS-XR 7.3.1.

Below, in the "List of sensor paths" section you find a list of YANG sensor path counters which
have changed the most while the issue occurred. 
Each line shows the name of the sensor path counter and the absolute amount that the sensor path counter 
has changed separated by the word CHANGE.
The sensor path counters are descriptive of the issue.

Perform the following two steps one after the other:

1. First, create a 'ISSUE', explain what is the issue with this router in a single paragraph. Be technical and specific.
2. Second, create a 'RESOLUTION', detailing your suggested next steps to fix the issue in a single paragraph. Be technical and specific.
Your response must only contain RFC8259 compliant JSON in the following format wrapped within [JSON] tags.
[JSON]
{
    "ISSUE": "your diagnosis",
    "RESOLUTION": "your resolution"
}
[JSON]
 
### List of sensor paths \n \n
$sensor_paths
""")
    
    _schema = {
        "type": "object",
        "properties": {
            "ISSUE": {"type": "string"},
            "RESOLUTION": {"type": "string"},
        },
        "required": [
            "ISSUE",
            "RESOLUTION"
        ]
    }

    def __init__(self, logger, llm : LlmInterface):
        super().__init__(logger, llm, self._schema)
        self._logger = logger
        self._data = None
                
    def run(self, sensor_paths):
        self._prompt = self._template.substitute(sensor_paths=sensor_paths)
        self._logger.info(f'Diagnose Sensor Paths Prompt:\n {self._prompt}')
        
        self._data = self.promptJsonResponse(self._prompt)
        
    def getDiagnosis(self):
        if self._data is not None:
            return f'ISSUE:\n{self._data["ISSUE"]}\n\nRESOLUTION:\n{self._data["RESOLUTION"]}'
        
        return ""
    
    def getIssue(self):
        return self._data["ISSUE"]        
    
    def getResolution(self):
        return self._data["RESOLUTION"]
                
    def get(self) -> str:
        if self._prompt is None:
            self._prompt = self._template.substitute(sensor_paths=self._sensor_paths)
        
        return self._prompt


### Create the Diagnosis
Use the first device from the selected changepoint, and use the correlating sensor paths as features to instantiate and run the Sensor Path Diagnosis prompt.

In [ ]:

sensor_path_diag = MdtDiagnosisPrompt(logger, llm)
sensor_path_diag.run(mdt_changepoint["Features"])

mdt_changepoint['Initial Diagnosis'] = sensor_path_diag.getDiagnosis()
utils.displayDictionary(mdt_changepoint)